In [2]:
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [6]:
model = ChatGoogleGenerativeAI(model='gemini-pro', convert_system_message_to_human= True)

In [7]:
response = model.invoke("Hello, world!").content

In [8]:
from langchain_core.output_parsers import StrOutputParser

In [9]:
parser = StrOutputParser()

In [10]:
chain = model | parser 

In [11]:
chain.invoke("Hello, world!")

''

In [12]:
from langchain_core.messages import HumanMessage

In [16]:
while True:
    message = input("Write your query")
    if message == "Bye":
        break
    else:
        print(chain.invoke([HumanMessage(content=message)]))

Hello there! How can I assist you today?
Yo, what's good?
